In [1]:
!python --version

Python 3.9.7


In [2]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

In [3]:
DATASET_PATH = Path(r'D:\fharookshaik\Documents\major_project\dataset')
TRAIN_CSV = os.path.join(DATASET_PATH,'train.csv')

In [4]:
train_df = pd.read_csv(TRAIN_CSV)
train_df.head()

,OCR,image,hero,villain,victim,other
0,Bernie or Elizabeth? Be informed.Compare them ...,covid_memes_18.png,NaN,NaN,NaN,"['bernie sanders', 'elizabeth warren']"
1,Extending the Brexit deadline until October 31...,covid_memes_19.png,NaN,['uk government'],NaN,NaN
2,kwai gkwa 0964 #nnevvy applause to Thais from ...,covid_memes_252.png,['thais'],NaN,NaN,['hong kong']
3,"So, I order this foce mask to protect ogainst ...",covid_memes_255.png,NaN,['china'],NaN,"['face mask', 'made in china', 'coronavirus']"
4,best candidate for JA 2020 joe biden Kamala ha...,covid_memes_20.png,['joe biden'],NaN,NaN,"['bernie sanders', 'kamala harris', 'tiktok']"


In [5]:
# Cleaning data
train_df['OCR'] = train_df['OCR'].fillna("")
train_df['hero'] = train_df['hero'].fillna({i: [] for i in train_df.index})
train_df['villain'] = train_df['villain'].fillna({i: [] for i in train_df.index})
train_df['victim'] = train_df['victim'].fillna({i: [] for i in train_df.index})
train_df['other'] = train_df['other'].fillna({i: [] for i in train_df.index})
train_df

,OCR,image,hero,villain,victim,other
0,Bernie or Elizabeth? Be informed.Compare them ...,covid_memes_18.png,[],[],[],"['bernie sanders', 'elizabeth warren']"
1,Extending the Brexit deadline until October 31...,covid_memes_19.png,[],['uk government'],[],[]
2,kwai gkwa 0964 #nnevvy applause to Thais from ...,covid_memes_252.png,['thais'],[],[],['hong kong']
3,"So, I order this foce mask to protect ogainst ...",covid_memes_255.png,[],['china'],[],"['face mask', 'made in china', 'coronavirus']"
4,best candidate for JA 2020 joe biden Kamala ha...,covid_memes_20.png,['joe biden'],[],[],"['bernie sanders', 'kamala harris', 'tiktok']"
...,...,...,...,...,...,...
5547,Trump could shoot someone on the Senate floor ...,memes_5039.png,[],['donald trump'],[],"['senate floor', 'republican']"
5548,MANY PEOPLE ASK ME WHY ALL MY SCHOOL RECORDS A...,memes_2635.png,[],[],[],"['school', 'university', 'joe biden']"
5549,my bes friend my mother consclence my therapis...,memes_1384.png,[],[],[],"['msnbc', 'bernie sanders', 'democratic party'..."
5550,THE N-WORD PASS Signed and approved by Beak Ob...,memes_944.png,[],[],[],['barack obama']


In [6]:
import re
import nltk.corpus
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fharookshaik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\fharookshaik\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
from nltk.corpus import stopwords

def sentences_to_nouns(sentence):
    # case normalization
    sentence = sentence.lower()

    # Remove unwanted chracters
    sentence = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", sentence)

    # Remove Stopwords
    stop = stopwords.words('english')
    sentence = " ".join([word for word in sentence.split() if word not in (stop)])

    # Tokenize sentence
    list_tokns = sentence.split()

    # Find nouns from sentence
    pos = nltk.pos_tag(list_tokns)
    is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    nouns = [word for (word,pos) in nltk.pos_tag(list_tokns) if is_noun(pos)]

    return nouns

In [8]:
entities = []
for idx,val in train_df.iterrows():
    entities.append(sentences_to_nouns(val.get('OCR')))

In [9]:
entities[0]

['bernie', 'elizabeth', 'issues', 'issue', 'memes']

In [10]:
sum = 0
for entity in entities:
    sum += len(entity)

sum

35963

In [11]:
unique_entities = []
for entitylist in entities:
    for entity in entitylist:
        if entity not in unique_entities:
            unique_entities.append(entity)

len(unique_entities)

10112

In [12]:
# enty_sent_dict = {}

# for entity in enty_sent_dict:
#     enty_sent_dict[entity] = []

# for idx,val in train_df.iterrows():
#     sentence = val.get('OCR').lower()
#     sentence = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", sentence)
#     for entity in entities:
#         print(entity,sentence)
#         if re.search(entity,sentence):
#             enty_sent_dict[entity] = enty_sent_dict[entity].append(sentence)

# enty_sent_dict      

In [13]:
import concurrent.futures
from tqdm import tqdm

# Find sentences according to entities
enty_sent_dict = {}

# for entity in enty_sent_dict:
#     enty_sent_dict[entity] = []

def find_enty_sent(entity):
    temp_sent_list = []
    for idx,val in train_df.iterrows():
        sentence = val.get('OCR').lower()
        sentence = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", sentence)
        if re.search(entity,sentence):
            temp_sent_list.append(sentence)
    enty_sent_dict[entity] = temp_sent_list
    return True

def run():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        result = list(tqdm(executor.map(find_enty_sent,unique_entities),total=len(unique_entities)))
    return result
    # for idx,res in enumerate(result):
    #     print(f'{idx} : {res}')
run()

 32%|███▏      | 3266/10112 [27:36<57:53,  1.97it/s]  
